## Ising Model GAN 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, "..")
import time
from pathlib import Path
data_path = Path("../../GetData/Python/Data")

## Load data

In [2]:
import tensorflow as tf
from input_pipeline import dataset_tfrecord_pipeline

class DataIterator:
    def __init__(self, datasets, temps):
        self.datasets = datasets
        self.temps = temps
    def __iter__(self):
        self.data_iterators = [iter(data) for data in self.datasets]
        return self
    
    def __next__(self):
        data_list = []
        temp_list = []
        for index, data_iterator in enumerate(self.data_iterators):
            data = next(data_iterator)
            data_list.append(data)
            temp = self.temps[index] * np.ones((data.shape[0], 1))
            temp_list.append(temp)
        temps = tf.concat(temp_list, axis=0)
        data = tf.concat(data_list, axis=0)
        return data, temps
        
def make_dataset(data_dir, temps, batch_size=100, flatten=False):
    if isinstance(data_dir, str):
        data_dir = Path(data_dir)
    
    assert batch_size % len(temps) == 0, "Batch size must be divisible by the number of temperatures"

    trainset = []
    testset = []
    for temp in temps:
        dataset = dataset_tfrecord_pipeline(data_dir / f"Data{temp:.1f}.tfrecord", flatten=flatten, batch_size=batch_size // len(temps))
        trainset.append(dataset)
        dataset = dataset_tfrecord_pipeline(data_dir / f"TestData{temp:.1f}.tfrecord", flatten=flatten, batch_size=batch_size // len(temps))
        testset.append(dataset)
    gen_test = DataIterator(testset, temps)
    test_dataset = tf.data.Dataset.from_generator(lambda: gen_test, output_signature = (tf.TensorSpec(shape=(None, 32, 32, 1), dtype=tf.float32), tf.TensorSpec(shape=(None, 1), dtype=tf.float32)))
    gen_train = DataIterator(trainset, temps)
    train_dataset = tf.data.Dataset.from_generator(lambda: gen_train, output_signature = (tf.TensorSpec(shape=(None, 32, 32, 1), dtype=tf.float32), tf.TensorSpec(shape=(None, 1), dtype=tf.float32)))
    return train_dataset, test_dataset

2024-03-27 16:31:03.157345: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
temps = [t for t in np.arange(2.0, 3.1, 0.1)]
batch_size = 110
trainset, testset = make_dataset(data_path, temps, batch_size=batch_size, flatten=False)

2024-03-27 16:31:10.930526: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1639] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 11551 MB memory:  -> device: 0, name: NVIDIA TITAN X (Pascal), pci bus id: 0000:04:00.0, compute capability: 6.1


In [4]:
for data, temps in trainset:
    print(data.shape, temps.shape)
    break

(110, 32, 32, 1) (110, 1)


In [5]:
from datetime import datetime
from pathlib import Path
# Get the current date and time
current_datetime = datetime.now()

# Format it to include date, hour, and minutes
formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M")

results = Path("results/" + formatted_datetime)
results.mkdir()

In [6]:
from tensorflow.keras import regularizers

def create_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 1), kernel_regularizer=regularizers.l2(0.01)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        tf.keras.layers.MaxPooling2D((2, 2)),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.01)),
        tf.keras.layers.Dense(1)
    ])
    return model

In [7]:
regression_model = create_model()
regression_model.compile(optimizer='adam', loss='mse', metrics=['mae'])
regression_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 15, 15, 32)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 15, 15, 32)        0         
                                                                 
 conv2d_1 (Conv2D)           (None, 13, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 6, 6, 64)          0         
 g2D)                                                            
                                                                 
 dropout_1 (Dropout)         (None, 6, 6, 64)          0

In [8]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=3)

regression_model.fit(trainset, epochs=10, validation_data=testset, callbacks=[early_stop])

Epoch 1/10


2024-03-27 16:31:14.177290: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape insequential/dropout/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer
2024-03-27 16:31:33.024764: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2024-03-27 16:31:34.750304: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2af2fb903160 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-27 16:31:34.750390: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA TITAN X (Pascal), Compute Capability 6.1
2024-03-27 16:31:34.958613: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-27 16:31:35.766661: I ./tensorflow/compiler/jit/device_compiler.h:186] Comp

   4999/Unknown - 113s 18ms/step - loss: 0.0978 - mae: 0.1055

2024-03-27 16:33:06.205415: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 12913853938430586633
2024-03-27 16:33:06.205480: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 16104199138889175154
2024-03-27 16:33:06.205739: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 8849016116601168032
2024-03-27 16:33:06.205823: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 1702279791957421076
2024-03-27 16:33:13.829436: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 11485228513473474509
2024-03-27 16:33:13.829578: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv item cancelled. Key hash: 5800440188162449564
2024-03-27 16:33:13.829945: I tensorflow/core/framework/local_rendezvous.cc:405] Local rendezvous recv 

5000/5000 [==============================] - 124s 20ms/step - loss: 0.0978 - mae: 0.1055 - val_loss: 0.0232 - val_mae: 0.0895
Epoch 2/10
5000/5000 [==============================] - 359s 44ms/step - loss: 0.0235 - mae: 0.0929 - val_loss: 0.0215 - val_mae: 0.0884
Epoch 3/10
5000/5000 [==============================] - 359s 44ms/step - loss: 0.0215 - mae: 0.0887 - val_loss: 0.0198 - val_mae: 0.0832
Epoch 4/10
5000/5000 [==============================] - 359s 44ms/step - loss: 0.0204 - mae: 0.0876 - val_loss: 0.0190 - val_mae: 0.0828
Epoch 5/10
5000/5000 [==============================] - 359s 44ms/step - loss: 0.0199 - mae: 0.0873 - val_loss: 0.0184 - val_mae: 0.0821
Epoch 6/10
5000/5000 [==============================] - 299s 44ms/step - loss: 0.0193 - mae: 0.0865 - val_loss: 0.0183 - val_mae: 0.0830
Epoch 7/10
5000/5000 [==============================] - 239s 32ms/step - loss: 0.0189 - mae: 0.0863 - val_loss: 0.0184 - val_mae: 0.0840
Epoch 8/10
5000/5000 [==============================

In [9]:
regression_model.save("results/regression_model")


INFO:tensorflow:Assets written to: results/regression_model/assets


INFO:tensorflow:Assets written to: results/regression_model/assets


In [10]:
loaded_model = tf.keras.models.load_model("results/regression_model")
